# Rerankers

This is a short notebook showing how to use Answer AI's rerankers library

In [3]:
%pip install "rerankers[transformers]" torch

  Obtaining dependency information for rerankers[transformers] from https://files.pythonhosted.org/packages/24/77/a0ea5b36221c89fbb7be9bf28075a57e829d89a72cbee0e69945b4524a7b/rerankers-0.5.3-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [4]:
import time
from utils import view
from rerankers import Reranker

/Users/markhneedham/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialising ReRanker

In [5]:
ranker = Reranker(
  model_name='mixedbread-ai/mxbai-rerank-large-v1',
  model_type='cross-encoder',
  device='mps'
)

Loading TransformerRanker model mixedbread-ai/mxbai-rerank-large-v1
No dtype set
Using dtype torch.float16
Loaded model mixedbread-ai/mxbai-rerank-large-v1
Using device mps.
Using dtype torch.float16.


In [6]:
%%time
sorted_rows = ranker.rank(
  query="I love you",
  docs=["I hate you", "I really like you", "You're not too bad"],
  doc_ids=[0,1,2]
)

CPU times: user 282 ms, sys: 184 ms, total: 466 ms
Wall time: 4.36 s


In [8]:
view(sorted_rows)

RankedResults(
    results=[
        Result(document=Document(text='I really like you', doc_id=1, metadata={}), score=-1.5400390625, rank=1),
        Result(document=Document(text="You're not too bad", doc_id=2, metadata={}), score=-2.8828125, rank=2),
        Result(document=Document(text='I hate you', doc_id=0, metadata={}), score=-4.30859375, rank=3)
    ],
    query='I love you',
    has_scores=True
)


## Reranking an Olympics dataset

In [9]:
import duckdb
con = duckdb.connect("olympics.duckdb")

In [12]:
view(
  con.query("FROM olympics SELECT index, text").limit(10)
)

┌───────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────
────┐
│ index │                                                     text                                                 
│
│ int64 │                                                   varchar                                                
│
├───────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────
────┤
│     0 │ The 2024 Olympics opened in Paris in spectacular style with thousands of athletes sailing along the River
…  │
│     1 │ Swapping a stadium for a waterway for the first time to open the "greatest show on Earth", the near 
four-h…  │
│     2 │ Blue, white and red fireworks had raised the Tricolore above Austerlitz Bridge before 6,800 athletes from
…  │
│     3 │ There were surprise performances through the ceremony, including a cabaret number from US 
singer-songwrite…  │
│     4 │ The day had started with

In [13]:
from search import Search
s = Search(con)

In [14]:
question = "What things went wrong?"
rows = s.vector_search(question, limit=10).fetchall()

In [15]:
view(rows)

[
    (
        'Given the miserable weather after what had been a sunny week in Paris until now, it seemed fitting that 
the storyline at the start of the ceremony was about the arrival of the Olympic flame in Paris not going according 
to plan.',
        15,
        0.5197256803512573
    ),
    ('A lot of the time it was brilliantly frenetic and occasionally emotional. ', 14, 0.4570101201534271),
    (
        "At times it was bizarre - one moment Lady Gaga surrounded by pink and black feathers was singing in 
French, the next Bangladesh's athletes were being introduced on their boat. ",
        13,
        0.45612600445747375
    ),
    (
        "The torchbearer did not get the memo about it not being in the Stade de France, and then Zinedine Zidane's
metro train broke down while he was transporting the torch.",
        16,
        0.44767796993255615
    ),
    (
        'The day had started with major disruption when the French train network was hit by arson attacks and heavy
ra

In [16]:
%%time
sorted_rows = ranker.rank(
  query=question,
  docs=[r[0] for r in rows],
  doc_ids=[r[1] for r in rows]
)

CPU times: user 175 ms, sys: 84.1 ms, total: 259 ms
Wall time: 786 ms


In [17]:
view(sorted_rows)

RankedResults(
    results=[
        Result(
            document=Document(
                text='The day had started with major disruption when the French train network was hit by arson 
attacks and heavy rain in the evening put paid to the original plan by artistic director Thomas Jolly to use the 
Parisian sun to "make the water sparkle". ',
                doc_id=4,
                metadata={}
            ),
            score=0.0321044921875,
            rank=1
        ),
        Result(
            document=Document(
                text="The torchbearer did not get the memo about it not being in the Stade de France, and then 
Zinedine Zidane's metro train broke down while he was transporting the torch.",
                doc_id=16,
                metadata={}
            ),
            score=-0.12066650390625,
            rank=2
        ),
        Result(
            document=Document(
                text='Given the miserable weather after what had been a sunny week in Paris unti